In [8]:
import os
import cv2
from concurrent.futures import ThreadPoolExecutor, as_completed

import numpy as np
from tqdm import tqdm

In [9]:
data_folder = r'.\model_test'

In [10]:
def letterbox(image, target_size=(640, 360), interpolation=cv2.INTER_AREA):
    h, w = image.shape[:2]
    target_w, target_h = target_size
    aspect_ratio = w / h
    target_aspect = target_w / target_h

    if aspect_ratio > target_aspect:
        scale = target_w / w
        new_w = target_w
        new_h = int(h * scale)
    elif aspect_ratio < target_aspect:
        scale = target_h / h
        new_h = target_h
        new_w = int(w * scale)
    else:
        new_h = target_h
        new_w = target_w

    img = cv2.resize(image, (new_w, new_h), interpolation=interpolation)

    return img



In [11]:
def process_input_image(filename, input_folder, resized_input_folder, interpolation=cv2.INTER_LINEAR):
    if filename.endswith(".jpg"):
        img_path = os.path.join(input_folder, filename)
        img = letterbox(cv2.imread(img_path) ,interpolation= interpolation)
        cv2.imwrite(os.path.join(resized_input_folder, filename), img)


In [12]:
def process_target_image(filename, target_folder, resized_target_folder, interpolation=cv2.INTER_NEAREST):
    if filename.endswith(".png"):
        target_path = os.path.join(target_folder, filename)
        target = letterbox(cv2.imread(target_path), interpolation=interpolation)
        cv2.imwrite(os.path.join(resized_target_folder, filename), target)


In [13]:
def resize_images(data_folder, output_size=(640, 360)):
    """
    Resize images from the 'input' and 'target' subfolders in data_folder.
    Resized images are saved in new folders 'input_resized' and 'target_resized'.
    The process uses multithreading via ThreadPoolExecutor and displays progress with tqdm.
    """
    # Define input and target folder paths
    input_folder = os.path.join(data_folder, 'input')
    target_folder = os.path.join(data_folder, 'target')


    if not os.path.exists(input_folder) or not os.path.exists(target_folder):
        print("The specified folders do not exist.")
        return

    # Create output folders for resized images
    resized_input_folder = os.path.join(data_folder, "input_resized")
    resized_target_folder = os.path.join(data_folder, "target_resized")
    os.makedirs(resized_input_folder, exist_ok=True)
    os.makedirs(resized_target_folder, exist_ok=True)

    # Get list of files for input and target images
    input_files = os.listdir(input_folder)
    target_files = os.listdir(target_folder)

    # Process input images using ThreadPoolExecutor with progress bar
    with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
        futures = [
            executor.submit(process_input_image, filename, input_folder, resized_input_folder)
            for filename in input_files if filename.endswith(".jpg")
        ]
        for _ in tqdm(as_completed(futures), total=len(futures), desc="Resizing input images"):
            pass

    # Process target images using ThreadPoolExecutor with progress bar
    with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
        futures = [
            executor.submit(process_target_image, filename, target_folder, resized_target_folder)
            for filename in target_files if filename.endswith(".png")
        ]
        for _ in tqdm(as_completed(futures), total=len(futures), desc="Resizing target images"):
            pass

    print("Resizing completed.")



In [14]:
if __name__ == "__main__":
    # Set your data folder path here
    resize_images(data_folder)


Resizing input images: 100%|██████████| 5000/5000 [00:33<00:00, 149.18it/s]

Resizing completed.
